# Evaluating RL Algorithms for Autonomous Navigation in Webots

[Github Repository](https://github.com/grace-ortiz/COGS188_Final)

## Group members

- Katie Chung
- Jiawei Gao
- Grace Ortiz
- Hsiang-An Pao

# Abstract

Autonomous driving has the potential to enhance road safety, reduce humans error, and improve traffic efficiency. However, a reliable decision-making in dynamic environments is still challenging. The objective of this study is to evaluate the performance of multiple reinforcement learning (RL) algorithms, which includes SARSA, Deep Q-Network (DQN), Proximal Policy Optimization (PPO), and Monte Carlo (MC), based on the high-fidelity driving simulator, Webots. These models process sensor data, including RGB images, LiDAR data, GPS coordinates, and speed measurements. Their goal is to navigate safely and optimizing performance at the same time. Our comparison focuses on some key aspects: handling of continuous vs. discrete state spaces, reward function effectiveness, and hyperparameter sensitivity. Results are evaluated based on reward convergence and policy efficiency. Our findings show that DQN demonstrated the best reward convergence, SARSA improved gradually but required more trials to converge, and Monte Carlo struggled with high variance and poor convergence, making it unsuitable for complex tasks. This work provides insights into the potential trade-offs between different RL approaches and also their applicability to autonomous driving, which can contribute to safer and more interpretable decision-making models in the future.

# Background

Self-driving cars could change transportation for the better by minimizing human-induced errors in driving, improving safety standards, and optimizing time-efficient driving. However, making reasonable decisions in a fast-moving and ever-changing situation is a huge difficulty. Traditional rule-based AI systems aren't sufficient when it comes to learning different driving conditions. They require advanced AI techniques and more specifically reinforcement learning (RL) algorithms, which enable the vehicle to optimally learn from experience[<sup>[1]</sup>](https://www.sciencedirect.com/science/article/pii/S0957417416303001?casa_token=rMm9DFyZgCMAAAAA:EczvI-ohrg_NX6XsU-PF3BgEKKIJDEX26VUfTIkIFhP_RTSpk_pvSx-1HMyGwq--Boh_o9bMWgg). This research aims to analyze the effectiveness of multiple RL variants, including SARSA, deep Q networks (DQN), Monte Carlo, and Proximal Policy Optimization (PPO) algorithms for one essential problem in self-driving functionality scenario, obstacle detection and evasion.

One fundamental barrier in AI vehicle operational decision-making is to find the optimal solution in a mixture of continuous and discrete dimensionalities space. These state representations have different efficiency of processing for different RL algorithms. Discrete state space represents for 2D SARSA and Monte Carlo (MC) has better performance, but loses a lot of details. In comparison, continuous action space DQN and PPO have better fine-tuned control on actions but are much more expensive computationally and require sophisticated policy function approximations[<sup>[2]</sup>](https://www.researchgate.net/profile/Dr-Anish-Pandey-2/publication/317101750_Mobile_Robot_Navigation_and_Obstacle_Avoidance_Techniques_A_Review/links/59266dad458515e3d45393b3/Mobile-Robot-Navigation-and-Obstacle-Avoidance-Techniques-A-Review.pdf).

Another critical aspect of autonomous driving is collision avoidance, a fundamental requirement for safe navigation. One such approach is the Collision Cone Method that has been adopted for defining safe movement regions around moving obstacles[<sup>[3]</sup>](https://ieeexplore.ieee.org/abstract/document/709600?casa_token=GcKiffriAj0AAAAA:GgeA985jE4U0L_1R9n8tkh6-RjT_j60BvuiVUeYw-yTfG2uElm3qF85BI8eJULafBhxp977v6Ik). Our project uses the current LiDAR data to detect obstacles in real-time, and the various RL models are analyzed in terms of their efficiency in avoiding collisions along with the driving tasks. Compared to traditional controllers, obstacle avoidance features have been improved with neural network-based RL techniques[<sup>[4]</sup>](https://link.springer.com/article/10.1007/s11370-021-00387-2), which deepens the understanding of the need for policy optimization without a harmonizing reinforcer.

In addition, autonomous vehicles have to follow lanes while driving, which requires them to find the most effective routes. It has been shown that some particular attention should be paid to reward shaping in reinforcement learning, especially in cases where optimization of the speed of rule-following should be counterbalanced[<sup>[5]</sup>](https://www.mdpi.com/2076-3417/13/14/8174). This project seeks to understand how reward structure impacts the results of learning with particular focus on the balance between exploration and exploitation within an RL model. Moreover, the tuning of hyperparameters (γ, α, and ε) is another factor that determines model performance and convergence, which is critical to the reliability of the decisions made.

By systematically testing and comparing SARSA, DQN, MC, and PPO, this study aims to see which RL model provides the most effective trade-off between safety, efficiency, and interpretability. The insights gained can contribute to the ongoing development of reliable and scalable reinforcement learning frameworks for self-driving applications.

# Problem Statement

Autonomous driving systems must make real-time decisions in dynamic environments while balancing safety, efficiency, and adherence to traffic rules. Traditional rule-based approaches struggle to generalize across diverse driving conditions, thus reinforcement learning (RL) is a promising alternative for self-driving applications. However, training RL models to drive safely and effectively remains a significant challenge due to the need for reliable evaluation metrics and the complexity of real-world driving scenarios.

This project aims to develop and compare reinforcement learning algorithms for self driving car simulations using WeBots. Our goal is to identify the model that acheives the highest overall performance in both reward convergence, the stabilization of cumulative rewards over training episodes, and policy efficiency, how well an agent maximizes rewards relative to the number of steps taken. The vehicle's actions can be evaluated using numerical metrics such as collision count, lane deviation, and distance covered from start. Additionally, each episode in the WeBots simulation can be analyzed for performance using well-defined criteria. This is also replicable, as the experiment can be conducted multiple times with different RL models and configurations to assess their effectiveness under various driving conditions.

Through this project, we aim to identify the RL algorithm that achieves the best trade-off between safety, rule adherence, and efficiency for autonomous driving.

# Data

The agent generated its own data through its intereaction with the Webots environment. Each observation from the agent consisted of various sensor readings:

| Sensor | Data Description | Raw Data Type | 
|---|---|---|
| Camera | BGRA image frame | 1D byte array |
| GPS | XYZ coordinates | 3D float array |
| LiDAR | SICK LMS 291 point cloud distance | 1D float array | 
| Gyro | 3-axis angular velocity| 3D float array |

The raw input data was processed into useable obserations for the state space. Due to the length of the data processing functions implemented, the process can be found in the in webots_env.py<a name="processing"></a>[<sup>[1]</sup>](#processingnote).

The BGRA image frames were processed by removing the alpha channel and converting the HSV color space [Figure 1], creating a lane mask to detect the yellow center line [Figure 2]. A region of interest focuses on the bottom half of the frame and Canny edge detection is used to determine the lane edges. Finally, a sliding window algorithm tracks lane positionsby identifying lane pixels in successive windows. Originally we attempted to use a combined mask to detect both the solid yellow lane line and the dotted white lane lines. However, due to the resolution of the simulated camera in Webots, the white line proved too difficult to detect consistently and was removed from the lane mask. The center of the lane was calculated by determining a fixed pixel distance from the yellow center lane line rather than the center of the two lane lines. 

<img src="final_imgs/frame.png" alt="WeBots Car Camera Frame" width="400">
<p align="left"><em>Figure 1: WeBots car camera frame</em></p>
<img src="final_imgs/lane_mask.png" alt="WeBots Car Camera Lane Mask" width="400">
<p align="left"><em>Figure 2: Assocated lane mask for WeBots frame</em></p>


The GPS x and y coordinates were extracted and passed to the state space, and the speed was extracted from the GPS and converted from m/s to km/h. The LiDAR data is processed by replacing any null values with a safe maxium distance of 100.0 km. The minimum obstacle distance is extracted and passed to the state space. Using the LiDAR's field of view (FOV) and the index of the minimum distance, the angle of the nearest obstacle is calculated and converted from radians to degrees. The gyroscope data is not used in the state space, however the x value is extracted and used to detect collisions by determining if the agent has flipped over. 

In [ ]:
# action space: [steering, speed]
self.action_space = spaces.Box(
    low=np.array([MIN_STEER_ANGLE, MIN_SPEED]), 
    high=np.array([MAX_STEER_ANGLE, MAX_SPEED]),
    dtype=np.float32
)

# state space 
self.state_space = spaces.Dict({
    "speed": spaces.Box(low=0, high=MAX_SPEED, shape=(1,), dtype=np.float32),  # gps speed
    "gps": spaces.Box(low=-np.inf, high=np.inf, shape=(2,), dtype=np.float32),  # (x, y) gps coordinates
    "lidar_dist": spaces.Box(low=0, high=100, shape=(1,), dtype=np.float32),  # distance to nearest obstacle
    "lidar_angle": spaces.Box(low=-90, high=90, shape=(1,), dtype=np.float32),  # angle to nearest obstacle
    "lane_deviation": spaces.Box(low=0, high=np.inf, shape=(1,), dtype=np.float32),  # pixels away from lane center
    "lane_mask": spaces.Box(low=0, high=1, shape=(64, 128, 1), dtype=np.uint8)  # binary mask for lane line (yellow line only)
})

The state space had 6 total variables: speed, gps, lidar_dist, lidar_angle, lane_deviation, lane_mask. The number of obervations was dependent on the number of episodes the agent trained and how many steps each epsiode lasted. Webots by default runs at 32ms per time step, meaning approximately 31 observations will be recorded per 1 second of simulation time. 


# Proposed Solution


To address the problem of training an autonomous self-driving vehicle in the Webots environment, we propose a solution based on four reinforcement learning algorithms: SARSA, Deep Q-Network (DQN), Proximal Policy Optimization (PPO), and Monte Carlo (MC) as a benchmark. Each algorithm is specifically adapted to handle the state observations provided by Webots (including speed, camera, lane deviation, and LIDAR readings) and is designed to optimize vehicle steering without accidents. The goal is to maximize cumulative rewards while ensuring adherence to traffic rules.




### Algorithmic Implementation Plan
1. SARSA (State-Action-Reward-State-Action)  -
SARSA will be used to learn an on-policy value function in a discretized state space. Continuous environment states will be discretized into bins. The agent selects actions using an epsilon-greedy strategy to balance exploration and exploitation. Q-values will be updated with Q(s,a) ← Q(s,a) + α [r + γQ(s’,a’) – Q(s,a)] ; This iterative update allows the agent to learn optimal policies in an on-policy manner. SARSA is reproducibly implemented using custom Python code integrated with Webots’ API , and results will be tested with various discretization granularities and learning rates.




2. DQN (Deep Q-Network) -
DQN will handle the continuous and high-dimensional state space using a neural network to approximate the Q-function. Neural network maps states to Q-values for each action. Experience Replay will store experiences in a replay buffer, sampled in mini-batches to stabilize training. Epsilon Decay helps educes exploration over time, shifting toward exploitation. The neural network and training loop will be implemented in PyTorch with reproducibility ensured via fixed random seeds and hyperparameter logging.  DQN is traditionally used for discrete action spaces, but it can be adapted for self-driving vehicle simulations, as some actions can be discretized.




3. PPO (Proximal Policy Optimization) -
PPO will directly optimize a parameterized policy for continuous control tasks, making it well-suited for vehicle navigation. This allows the agent to learn smooth and precise control policies, such as gradually adjusting the steering angle or smoothly accelerating/decelerating.[<sup>[7]</sup>](https://aryanjha.medium.com/creating-a-self-driving-car-simulation-977bed8f49b4) Policy Network allows Nneural networks trained to output action distributions and value estimates. THe clipped surrogate ensures stable policy updates by preventing excessively large updates. Generalized Advantage Estimation (GAE)reduces variance in policy gradient estimates with multiple epoch updates that increases sample efficiency.




4. MC (Monte Carlo) -
MC will be implemented as a benchmark to compare against the other RL methods. The algorithm will collect complete episode trajectories, compute returns at the end of each episode, and update Q-values based on the average return for each state-action pair. The Monte Carlo method will serve as the benchmark. While slower to converge, MC provides unbiased estimates of the value function and will be used to evaluate the relative performance of SARSA, DQN, and PPO in terms of convergence speed and cumulated rewards.




### Testing and Evaluation
All algorithms will be tested in the Webots simulation using the following metrics:




Episode Reward: The total accumulated reward per episode.
<br>
Convergence: Convergence across different RL algorithms would be compared to see how well each learned.
<br>
Success/Safety Metrics (TBD by training progress): The percentage of episodes where the vehicle reaches its goal without collisions. The number of collisions and lane deviation during stimlation.


# Evaluation Metrics for Reinforcement Learning

## **1. Reward Convergence**
Reward convergence refers to the stabilization of cumulative rewards over training episodes, indicating that the agent has learned an optimal policy. To track this, we use the **moving average of rewards**:

$$
R_t^{\text{avg}} = \frac{1}{N} \sum_{i=t-N}^{t} R_i
$$

where:
- $R_i$ is the total reward in episode $i$.
- $R_t^{\text{avg}}$ is the moving average reward at episode $t$.
- $N$ is the window size.

A model is considered to have converged when the variance of $R_t^{\text{avg}}$ falls below a predefined threshold $\epsilon$:

$$
\text{Var}(R_t^{\text{avg}}) < \epsilon
$$

---
## **2. Policy Efficiency**
Policy efficiency measures how well an agent maximizes rewards relative to the number of steps taken. It is computed as:

$$
\text{Efficiency} = \frac{\sum_{t=1}^{T} R_t}{T}
$$

where:
- $R_t$ is the reward at step $t$.
- $T$ is the total number of steps.

If we also consider the number of actions taken, we define the **reward efficiency ratio**:

$$
\text{Reward Efficiency Ratio} = \frac{\sum_{t=1}^{T} R_t}{\sum_{t=1}^{T} A_t}
$$

where $A_t$ is the number of actions taken at step $t$.

Higher efficiency values indicate that the agent is maximizing rewards while minimizing unnecessary steps or actions

# Results
Our analysis highlights sample efficiency as the most crucial factor in determining the effectiveness of reinforcement learning algorithms for high-dimensional tasks such as autonomous driving. We compared Monte Carlo (MC), SARSA, and Deep Q-Network (DQN) by evaluating how quickly each algorithm converged to stable driving behavior.

## #Deep Q-Network (DQN) Performance
DQN demonstrated the best sample efficiency, showing signs of convergence within approximately 500 trials. The use of experience replay and function approximation allowed DQN to generalize across states, enabling faster learning and producing more consistent performance over time. Although DQN still exhibited fluctuations in performance, its reward trend remained relatively stable compared to Monte Carlo and SARSA. Policy efficiency, measured as reward per step, showed less degradation over time, indicating that DQN retained learned behaviors better than the other methods.

<img src="final_imgs/dqn_training_metrics.png" alt="" width="400">
<p align="left"><em>DQN Training Metrics</em></p>

### SARSA Performance
SARSA failed to show consistent improvement or fully converge within a reasonable number of episodes. The moving average of rewards showed that SARSA learned more slowly, and the efficiency fluctuated significantly over time. While the model initially improved, it struggled to maintain performance, suggesting that it was more prone to unlearning or adapting too slowly to policy refinements. The increased variance in rewards later in training suggests that SARSA may be overly sensitive to small changes in the environment, leading to more erratic behavior.

<img src="final_imgs/sarsa_training_metrics.png" alt="" width="400">
<p align="left"><em>SARSA Training Metrics</em></p>

### Monte Carlo Performance
Monte Carlo exhibited the highest variance in rewards, with large spikes in episodic returns but no clear trend toward convergence. While it occasionally produced very high rewards, the policy remained unstable and inconsistent across trials. Unlike SARSA and DQN, Monte Carlo does not rely on bootstrapping, which likely contributed to its inefficiency in this task. The inability to use temporal difference updates made Monte Carlo significantly less sample efficient, requiring many more episodes to refine its policy effectively. The final results indicate that Monte Carlo struggled to adapt to the complexity of the environment, making it the least effective method for learning a stable driving policy.

<img src="final_imgs/monte_carlo_g.png" alt="" width="400">
<p align="left"><em>Monte Carlo Results</em></p>


### Differing Rewards Functions
have graph for sarsa and mc with diff reward functions



### Hyperparameter Exploration

First, let's explore the the learning Rate (Alpha). A higher Alpha (0.1) leads to faster learning but can cause instability, as seen in the first graph. A lower Alpha (0.05) provides more stability but slower convergence. An intermediate Alpha (0.08) seems to offer a good balance, leading to stable and significant reward increases.

Next, a higher Gamma (0.99) focuses more on long-term rewards, which is beneficial for tasks requiring long-term planning. The slight reduction in Gamma (0.95) in the second graph might be suitable for tasks where short-term rewards are more critical. For exploration rate, the decay pattern of Epsilon is crucial for balancing exploration and exploitation. A controlled decay, as seen in the third graph, helps in gradually shifting from exploration to exploitation, leading to better policy learning.

In summary, hyperparameter tuning in SARSA involves balancing the learning rate, discount factor, and exploration rate to achieve stable and optimal learning. The third graph with Alpha: 0.08 and Gamma: 0.99 demonstrates a well-tuned set of hyperparameters, leading to effective and stable learning progress.



<img src="final_imgs/sarsa_a01.png" alt="" width="400">
<p align="left"><em>Figure 1: SARSA Higher Learning Rate& Discount</em></p>
<img src="final_imgs/sarsa_a05.png" alt="" width="400">
<p align="left"><em>Figure 2: SARSA Lower Learning Rate& Discount</em></p>
<img src="final_imgs/SARSA_g.png" alt="SARSA best performing" width="400">
<p align="left"><em>Figure 3: SARSA Best Performing</em></p>



# Discussion




### Interpreting the result




OK, you've given us quite a bit of tech informaiton above, now its time to tell us what to pay attention to in all that.  Think clearly about your results, decide on one main point and 2-4 secondary points you want us to understand. Highlight HOW your results support those points.  You probably want 2-5 sentences per point.

### Sample efficiency (delete header later)
The sample efficiency analysis revealed clear differences in how each algorithm handles learning in a complex, high-dimensional environment like autonomous driving. Deep Q Network (DQN) demonstrated superior learning efficiency due to its experience replay and Q-function approximation, allowing it to learn faster than the compared algorithms. The use of target networks helps to reduce oscillations, preventing instability. Unlike Monte Carlo and SARSA, DQN can handle larger discrete state spaces  by generalizing across similar states through function approximation. This is in contrast to SARSA, which must take each action to understand its outcome, severely impacting its performance in more complex environments due to its on-policy nature but makes it more stable than Monte Carlo. However, Monte Carlo updates episodically, it results in poor sample efficiency. Since it relies on visiting full trajectories before it updates an extremely large number of episodes are required to cover large state spaces. Although PPO was not successfully trained in this environment due to technical issues, it was expected to perform well given its ability to generalize to large *continuous* state spaces, offering a sharp contrast to other algorithms tested. When dealing with complex continous state spaces, algorithms that can approximate rewards tend to perform better since the entire state space does not need to be explored. 


### Hyperparameter
The results highlight the critical role of hyperparameter tuning in the SARSA algorithm's performance. The comparison of different Alpha and Gamma values demonstrates that a moderate learning rate (Alpha: 0.08) and a high discount factor (Gamma: 0.99) lead to stable and effective learning, as evidenced by the steady increase in total reward in the third graph. This supports the importance of balancing exploration and exploitation, as controlled by Epsilon decay, to achieve optimal policy learning. The findings highlights that careful hyperparameter tuning is essential for maximizing the reinforcement learning algorithm's performance, particularly in tasks requiring long-term reward optimization. These insights are valuable for future work when dealing with a more complex situation.



### Limitations

Due to time and computational constraints, we were unable to run enough trials for the algorithms to fully converge. This was particularly evident with SARSA, which showed gradual improvement over 1000 trials but likely required significantly more episodes to reach stability. Similarly, DQN’s early signs of convergence around 500 trials suggested it could have achieved greater stability with additional training. The large and complex state space of the autonomous driving environment further amplified this issue. Algorithms like Monte Carlo, which already suffer from poor sample efficiency, were particularly impacted as it failed to converge — or show any indication of learning — within the limited trial count. Due to time constraints, we were unable to fully explore the hyperparameter space for each algorithm. Parameters such as the learning rate, discount factor, exploration rate, action bins, or state bins could significantly impact performance. With more time, extensive hyperparameter optimization could potentially lead to better convergence and stability. 

The large state space meant that certain states and actions were rarely visited during training. This uneven coverage could have biased the learning process, particularly for Monte Carlo, which struggles with sparse state visitation. Due to the extensive amount of time training the agent consumes, fewer results wer able to be collected, limiting our generalizability and statistical reliability. Furthermore, processing vehicle sensor data, especially camera frames, posed significant challenges due to the quality of sensor output. Finally, the inability to successfully train PPO meant we could not evaluate its potential advantage in continuous state spaces, limiting the completeness of our comparison


### Future work
To address the limitations observed in this study, future work will focus on improving the performance of reinforcement learning algorithms and enhancing sample efficiency. First, extending the training duration and increasing the number of trials for each algorithm will be crucial. Algorithms like SARSA and DQN showed gradual improvement over time, but they did not fully converge within the limited trial count. By running longer experiments with more trials, we can allow these algorithms to reach greater stability and optimal performance. To enable this, we would also need to upgrade our computational power. Using computers with powerful GPUs is essential, as running experiments on MacBooks proves to be too inefficient for such complex environments and tasks.

If more time is available in the future, we would conduct a more extensive hyperparameter search, focusing on parameters such as the learning rate, exploration rate, and state/action bins. A thorough exploration of the hyperparameter space could help identify the optimal settings that would allow for faster and more stable convergence. Additionally, if we can create a variety of environments in Webots, such as shorter or differently structured paths, it would provide a broader and more challenging set of conditions for the algorithms to learn from. Setting up the environment took a significant amount of time early in the project, so in future work, we plan to test algorithms across different scenarios. This will help evaluate the adaptability and performance of the reinforcement learning algorithms in various settings.

Lastly, exploring the potential of PPO in continuous state spaces is an area worth revisiting. Since we were new to working with PPO, it was not successfully trained in this study. It would be helpful if more people, with additional time, could analyze why it struggled and help modify its configuration for better performance. Adjusting its reward structure or training process could reveal ways to successfully integrate PPO into this environment.




### Ethics & Privacy




Ethical concerns in the development and deployment of self-driving vehicles are multifaceted and require careful consideration, with safety emerging as the foremost priority. Self-driving vehicles rely heavily on machine learning models to interpret their surroundings, make real-time decisions, and navigate complex environments. However, these models are not infallible and may encounter unpredictable or edge-case scenarios that could lead to accidents or unsafe outcomes. Unlike human drivers, AI systems lack personal accountability, raising significant questions about responsibility and liability in the event of failures. For instance, if an accident occurs due to a flaw in the ML model, determining who is at fault—whether it be the manufacturer, the software engineers, the vehicle owner, or even the regulatory bodies—becomes a complex legal and ethical challenge.[<sup>[8]</sup>](https://aryanjha.medium.com/creating-a-self-driving-car-simulation-977bed8f49b4).
This issue is further exacerbated if our model contributes to such failures, underscoring the need for robust testing, transparency, and accountability mechanisms.




One of the most profound ethical dilemmas in self-driving technology arises in unavoidable accident scenarios, often referred to as the "trolley problem."[<sup>[9]</sup>](https://montrealethics.ai/the-ethical-considerations-of-self-driving-cars/). In these situations, the AI system may be forced to choose between different harmful outcomes, such as prioritizing the safety of its passengers over pedestrians or other drivers. This raises difficult questions about how ethical guidelines should be programmed into the system. Should the vehicle prioritize the greater good, minimize harm, or protect its occupants at all costs?. These decisions are not only technically challenging to implement but also philosophically contentious, as they require engineers to encode moral principles into algorithms. This challenge highlights the broader difficulty of defining and standardizing ethical guidelines for AI systems, particularly in a way that aligns with societal values and legal frameworks.




Bias in machine learning models is another critical ethical consideration that cannot be overlooked. The performance of self-driving systems is heavily dependent on the quality and diversity of the training data. If the dataset used to train the ML models lacks representation of a wide range of pedestrian appearances, road conditions, weather scenarios, or cultural contexts, the AI may struggle to make fair and accurate decisions in real-world environments. For example, a model trained primarily on data from urban areas with specific demographics might underperform in rural settings or fail to recognize pedestrians with diverse physical characteristics.[<sup>[10]</sup>](https://bytes.scl.org/self-driving-cars-ethics-the-trolley-problem/)This could lead to unsafe or discriminatory outcomes, disproportionately affecting certain groups. To mitigate this risk, it is essential to ensure that training datasets are comprehensive, inclusive, and rigorously tested across diverse environments. Additionally, ongoing monitoring and updating of the models are necessary to address biases that may emerge over time.




Privacy is another significant ethical concern in the development of self-driving vehicles. These systems collect and process vast amounts of data, including real-time location information, passenger behavior, and even personal or vehicle-specific details. While this data is crucial for improving the performance and safety of AI systems, it also poses a risk to user privacy if not handled responsibly. Unauthorized access, data breaches, or misuse of this information could lead to serious privacy violations. To address this, we would ensure that all data collected is anonymized and securely stored, minimizing the risk of privacy leaks. Furthermore, clear policies and protocols must be established to govern data collection, usage, and sharing, ensuring compliance with privacy regulations and building trust with users.








### Conclusion




Reiterate your main point and in just a few sentences tell us how your results support it. Mention how this work would fit in the background/context of other work in this field if you can. Suggest directions for future work if you want to.


# Footnotes

<a name="duguleananote"></a>1.^: Duguleana, M., & Mogan, G. (2016). Neural networks based reinforcement learning for mobile robots obstacle avoidance. Expert Systems With Applications, 62, 104–115. https://www.sciencedirect.com/science/article/pii/S0957417416303001?casa_token=rMm9DFyZgCMAAAAA:EczvI-ohrg_NX6XsU-PF3BgEKKIJDEX26VUfTIkIFhP_RTSpk_pvSx-1HMyGwq--Boh_o9bMWgg<br>

<a name="pandeynote"></a>2.^: Pandey, A., Pandey, S., & Parhi, D. R. (2017). Mobile Robot Navigation and Obstacle Avoidance Techniques: A Review. International Robotics & Automation Journal, 2(3), 00023. https://www.researchgate.net/profile/Dr-Anish-Pandey-2/publication/317101750_Mobile_Robot_Navigation_and_Obstacle_Avoidance_Techniques_A_Review/links/59266dad458515e3d45393b3/Mobile-Robot-Navigation-and-Obstacle-Avoidance-Techniques-A-Review.pdf<br>

<a name="chakravarthynote"></a>3.^: Chakravarthy, A., & Ghose, D. (1998). Obstacle avoidance in a dynamic environment: A collision cone approach. IEEE Transactions on Systems, Man, and Cybernetics—Part A: Systems and Humans, 28(5), 562–574. https://ieeexplore.ieee.org/abstract/document/709600?casa_token=GcKiffriAj0AAAAA:GgeA985jE4U0L_1R9n8tkh6-RjT_j60BvuiVUeYw-yTfG2uElm3qF85BI8eJULafBhxp977v6Ik<br>

<a name="choinote"></a>4.^: Choi, J., Lee, G., & Lee, C. (2021). Reinforcement learning-based dynamic obstacle avoidance and integration of path planning. Intelligent Service Robotics, 14, 663–677. https://link.springer.com/article/10.1007/s11370-021-00387-2<br>

<a name="almazroueinote"></a>5.^: Almazrouei, K., Kamel, I., & Rabie, T. (2023). Dynamic Obstacle Avoidance and Path Planning through Reinforcement Learning. Applied Sciences, 13(8174). https://www.mdpi.com/2076-3417/13/14/8174<br>

<a name="processingnote"></a>6.[^](#processing): https://github.com/grace-ortiz/COGS188_Final/blob/main/webots/controllers/webots_env.py<br>

<a name="self_driving_sim_note"></a>7.^: Aryan Jha. (2021). Creating a self-driving car simulation. Medium. https://aryanjha.medium.com/creating-a-self-driving-car-simulation-977bed8f49b4<br>

<a name="self_driving_ethics_note"></a>8.^: Gunkel, D. J. (2021). The Ethics of Autonomous Vehicles. Springer. https://doi.org/10.1007/s13347-021-00464-5<br>

<a name="self_driving_ethics_article_note"></a>9.^: Zac Amos. (2021). The Ethical Considerations of Self-Driving Cars. https://montrealethics.ai/the-ethical-considerations-of-self-driving-cars/<br>

<a name="trolley_problem_ethics_note"></a>10.^: Burak Haylamaz. (2020). Self-driving Cars Ethics: The Trolley Problem. https://bytes.scl.org/self-driving-cars-ethics-the-trolley-problem/<br>